In [1]:
from pulp import *
import itertools

### Question 7

In [2]:
#Create a list of all the plants
Plants = ["P1","P2","P3","P4"]

#Create a dictionary of the minimum quantity to each plant
PlantsConst = {"P1":80, "P2":100, "P3":70, "P4":130}

#Create a list of all the mines
Mines = ["M1","M2"]

#Create a dictionary of the maximum quantity from each mine
MinesConst = {"M1":230, "M2":150}

#Create a list of all the plant-mines pairs
Plant_Mines = ['{}_{}'.format(x[0], x[1]) for x in itertools.product(Plants, Mines)]

#Create a dictionary for amount used of each raw material for each product
Transportation_Costs = {"M1": {"P1":65, "P2":40, "P3":30, "P4":15},
                        "M2": {"P1":10, "P2":35, "P3":40, "P4":60}}

# Create the 'prob' variable to contain the problem data
prob = LpProblem("Coal Mines", LpMinimize)

# Create the Variable for percentages of raw gas types in fuel mixes
PM_vars = LpVariable.dicts("var",Plant_Mines,lowBound=0,upBound=None,cat=LpContinuous)

# The objective function is added to 'prob' first
prob += lpSum([PM_vars[i] * Transportation_Costs[i[3:5]][i[0:2]] for i in Plant_Mines]), 'Total cost of Transportation'

# We can enter the constraints that relate to the plants
for r in Plants:
    prob += lpSum([PM_vars['{}_{}'.format(r, i)] for i in Mines]) >= PlantsConst[r], 'Plant_{}'.format(r)

# We can enter the constraints that relate to the mines
for r in Mines:
    prob += lpSum([PM_vars['{}_{}'.format(i, r)] for i in Plants]) <= MinesConst[r], 'Mine_{}'.format(r)
    
# The problem data is written to an .lp file
prob.writeLP("PlantsMines.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen
print("Total Cost = ", value(prob.objective))

for constraint in prob.constraints:
        print(prob.constraints[constraint].name, prob.constraints[constraint].value())
        print(prob.constraints[constraint].name, prob.constraints[constraint].constant)

Status: Optimal
var_P1_M1 = 0.0
var_P1_M2 = 80.0
var_P2_M1 = 30.0
var_P2_M2 = 70.0
var_P3_M1 = 70.0
var_P3_M2 = 0.0
var_P4_M1 = 130.0
var_P4_M2 = 0.0
Total Cost =  8500.0
Plant_P1 0.0
Plant_P1 -80
Plant_P2 0.0
Plant_P2 -100
Plant_P3 0.0
Plant_P3 -70
Plant_P4 0.0
Plant_P4 -130
Mine_M1 0.0
Mine_M1 -230
Mine_M2 0.0
Mine_M2 -150


In [3]:
prob

Coal Mines:
MINIMIZE
65*var_P1_M1 + 10*var_P1_M2 + 40*var_P2_M1 + 35*var_P2_M2 + 30*var_P3_M1 + 40*var_P3_M2 + 15*var_P4_M1 + 60*var_P4_M2 + 0
SUBJECT TO
Plant_P1: var_P1_M1 + var_P1_M2 >= 80

Plant_P2: var_P2_M1 + var_P2_M2 >= 100

Plant_P3: var_P3_M1 + var_P3_M2 >= 70

Plant_P4: var_P4_M1 + var_P4_M2 >= 130

Mine_M1: var_P1_M1 + var_P2_M1 + var_P3_M1 + var_P4_M1 <= 230

Mine_M2: var_P1_M2 + var_P2_M2 + var_P3_M2 + var_P4_M2 <= 150

VARIABLES
var_P1_M1 Continuous
var_P1_M2 Continuous
var_P2_M1 Continuous
var_P2_M2 Continuous
var_P3_M1 Continuous
var_P3_M2 Continuous
var_P4_M1 Continuous
var_P4_M2 Continuous